# ACTUS OPTNS Contract: European Call and Put Options

This notebook demonstrates the **Options (OPTNS)** contract type from the ACTUS standard.

## What is an OPTNS Contract?

An Options contract gives the holder the right (but not obligation) to:
- **Call Option**: Buy the underlier at strike price
- **Put Option**: Sell the underlier at strike price

Exercise types:
- **European**: Exercise only at maturity
- **American**: Exercise anytime before maturity

## Example 1: European Call Option on Stock

- Current Stock Price: $100
- Strike Price: $105
- Maturity: 1 year
- Option Type: Call (right to buy)

In [ ]:
from jactus.contracts import create_contract
from jactus.core import (
    ContractAttributes,
    ContractType,
    ContractRole,
    ActusDateTime,
)
from jactus.observers import ConstantRiskFactorObserver, MockChildContractObserver
from jactus.core import ContractState, ContractPerformance
import jax.numpy as jnp

## Step 1: Create Underlier (Stock Contract)

In [ ]:
# Contract dates
status_date = ActusDateTime(2024, 1, 1, 0, 0, 0)
maturity_date = ActusDateTime(2025, 1, 1, 0, 0, 0)  # 1 year

# Create stock (underlier) state
stock_state = ContractState(
    tmd=maturity_date,
    sd=status_date,
    nt=jnp.array(100.0, dtype=jnp.float32),  # Current stock price
    ipnr=jnp.array(0.0, dtype=jnp.float32),
    ipac=jnp.array(0.0, dtype=jnp.float32),
    feac=jnp.array(0.0, dtype=jnp.float32),
    nsc=jnp.array(1.0, dtype=jnp.float32),
    isc=jnp.array(1.0, dtype=jnp.float32),
    prf=ContractPerformance.PF,
)

# Register stock with child observer
child_observer = MockChildContractObserver()
child_observer.register_child("STOCK-AAPL", state=stock_state, events=[])

print(f"Stock (Underlier) Price: ${float(stock_state.nt):.2f}")

## Step 2: Create European Call Option

In [ ]:
# Create call option attributes
call_attrs = ContractAttributes(
    contract_id="CALL-AAPL-105",
    contract_type=ContractType.OPTNS,
    contract_role=ContractRole.RPA,  # Long position
    status_date=status_date,
    maturity_date=maturity_date,
    option_type="C",  # Call option
    option_strike_1=105.0,  # Strike price
    option_exercise_type="E",  # European
    currency="USD",
    contract_structure='{"Underlier": "STOCK-AAPL"}',  # Reference to stock
)

# Create risk factor observer
rf_observer = ConstantRiskFactorObserver(constant_value=0.05)

# Create option contract
call_option = create_contract(call_attrs, rf_observer, child_observer)

print(f"\nCall Option Created:")
print(f"  Strike Price: ${call_attrs.option_strike_1}")
print(f"  Type: European Call")
print(f"  Maturity: 1 year")

## Step 3: Simulate Call Option Payoff

At maturity, the payoff is:
- Call: max(Stock Price - Strike Price, 0)

In [ ]:
# Simulate option
call_result = call_option.simulate(rf_observer)

print("\nCall Option Simulation Results:")
print(f"Events generated: {len(call_result.events)}")

if call_result.events:
    for event in call_result.events:
        payoff = float(event.payoff)
        print(f"\nEvent: {event.event_type.value}")
        print(f"Time: {event.event_time.year}-{event.event_time.month:02d}-{event.event_time.day:02d}")
        print(f"Payoff: ${payoff:.2f}")
        
        # Calculate intrinsic value
        stock_price = float(stock_state.nt)
        strike = call_attrs.option_strike_1
        intrinsic = max(stock_price - strike, 0)
        print(f"\nCalculation:")
        print(f"  Stock Price: ${stock_price:.2f}")
        print(f"  Strike Price: ${strike:.2f}")
        print(f"  Intrinsic Value: max(${stock_price:.2f} - ${strike:.2f}, 0) = ${intrinsic:.2f}")
        print(f"  Status: {'In the Money' if intrinsic > 0 else 'Out of the Money'}")

## Example 2: European Put Option

Put option gives the right to **sell** at strike price

In [ ]:
# Create put option attributes
put_attrs = ContractAttributes(
    contract_id="PUT-AAPL-95",
    contract_type=ContractType.OPTNS,
    contract_role=ContractRole.RPA,
    status_date=status_date,
    maturity_date=maturity_date,
    option_type="P",  # Put option
    option_strike_1=95.0,  # Strike price
    option_exercise_type="E",  # European
    currency="USD",
    contract_structure='{"Underlier": "STOCK-AAPL"}',
)

# Create put option
put_option = create_contract(put_attrs, rf_observer, child_observer)

print(f"\nPut Option Created:")
print(f"  Strike Price: ${put_attrs.option_strike_1}")
print(f"  Type: European Put")
print(f"  Maturity: 1 year")

# Simulate
put_result = put_option.simulate(rf_observer)

print(f"\n Put Payoff: max(Strike - Stock, 0) = max(${put_attrs.option_strike_1} - ${float(stock_state.nt)}, 0)")
if put_result.events:
    payoff = float(put_result.events[0].payoff)
    print(f"  = ${abs(payoff):.2f}")

## Example 3: Payoff Diagram for Different Stock Prices

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Range of possible stock prices at maturity
stock_prices = np.linspace(80, 120, 100)
call_strike = 105.0
put_strike = 95.0

# Calculate payoffs
call_payoffs = np.maximum(stock_prices - call_strike, 0)
put_payoffs = np.maximum(put_strike - stock_prices, 0)

# Create plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Call option payoff
ax1.plot(stock_prices, call_payoffs, linewidth=2, color='green', label='Call Payoff')
ax1.axvline(x=call_strike, color='red', linestyle='--', label=f'Strike = ${call_strike}')
ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax1.fill_between(stock_prices, 0, call_payoffs, alpha=0.3, color='green')
ax1.set_xlabel('Stock Price at Maturity ($)', fontsize=11)
ax1.set_ylabel('Option Payoff ($)', fontsize=11)
ax1.set_title('European Call Option Payoff', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Put option payoff
ax2.plot(stock_prices, put_payoffs, linewidth=2, color='red', label='Put Payoff')
ax2.axvline(x=put_strike, color='green', linestyle='--', label=f'Strike = ${put_strike}')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.fill_between(stock_prices, 0, put_payoffs, alpha=0.3, color='red')
ax2.set_xlabel('Stock Price at Maturity ($)', fontsize=11)
ax2.set_ylabel('Option Payoff ($)', fontsize=11)
ax2.set_title('European Put Option Payoff', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Payoff Diagrams:")
print(f"  • Call: Profitable when stock > ${call_strike} (in the money)")
print(f"  • Put: Profitable when stock < ${put_strike} (in the money)")

## Key Observations

The OPTNS contract type implements:

1. **Option Payoff Logic**:
   - Call: max(S - K, 0)
   - Put: max(K - S, 0)

2. **Exercise Types**:
   - European (E): Exercise only at maturity
   - American (A): Can exercise anytime (not shown here)

3. **Underlier Reference**: Links to child contract via contract_structure

4. **ACTUS Compliance**: Follows ACTUS v1.1 specification for derivative contracts

## References

- ACTUS Specification v1.1, Section 7.12 - Options (OPTNS)
- [ACTUS Technical Specification](https://www.actusfrf.org/)